In [3]:
pip install opencv-python matplotlib

Note: you may need to restart the kernel to use updated packages.


In [4]:
from matplotlib import pyplot as plt
import cv2
import numpy as np
import concrete.numpy as cnp

In [5]:
%matplotlib inline
def imshow(img):
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_GRAY2RGB))
    plt.show()

In [6]:
def imread_black_and_white(path):
    image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    return np.where(image == 255, 1, image)

In [7]:
def read_ap_img(path, shape):
    ap = np.zeros(shape, dtype=np.uint8)
    
    with open(path, "r") as f:
        n_points = int(f.readline())
        for i in range(n_points):
            x, y = f.readline().split()
            x = int(x)
            y = int(y)
            ap[x][y] = 255
        
    return ap

In [8]:
def read_ap_indices(path):
    points = []
    
    with open(path, "r") as f:
        n_points = int(f.readline())
        for i in range(n_points):
            x, y = f.readline().split()
            x = int(x)
            y = int(y)
            for i in range(1, 7):
                points.append([x*i, y])
            
    # Transform it in a way to be used for numpy arrays like this: x[ap_indices]
    points = tuple(np.array(points).T.tolist())
        
    return points

In [9]:
64 * 512 * 6

196608

In [10]:
ap = read_ap_img("points.txt", (64, 512))

In [11]:
ap_indices = read_ap_indices("points.txt")

In [12]:
DATA_DIR = "../data/Output"

In [13]:
iris_code = imread_black_and_white(f"{DATA_DIR}/IrisCodes/0000_000_code.bmp")
mask = imread_black_and_white(f"{DATA_DIR}/NormalizedMasks/0000_000_mano.bmp")

In [14]:
iris_code2 = imread_black_and_white(f"{DATA_DIR}/IrisCodes/0000_002_code.bmp")
mask2 = imread_black_and_white(f"{DATA_DIR}/NormalizedMasks/0000_003_mano.bmp")

In [15]:
total_mask = mask & mask2 & ap
total_mask_big = np.array(total_mask.tolist() * 6)

In [17]:
total_mask_big.shape

(384, 512)

In [20]:
384 * 512, 6*256

(196608, 1536)

In [23]:
ic_ap = iris_code[ap_indices]
ic2_ap = iris_code2[ap_indices]
m_ap = total_mask_big[ap_indices]

In [24]:
match_iris(iris_code, iris_code2, total_mask_big), np.sum(total_mask_big)

(326, 882)

In [25]:
match_iris(ic_ap, ic2_ap, m_ap), np.sum(m_ap)

(157, 378)

In [26]:
from itertools import combinations, combinations_with_replacement

In [27]:
ij = [(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3)]

In [29]:
for i, j in ij:
    iris_code = imread_black_and_white(f"{DATA_DIR}/IrisCodes/000{i}_00{j}_code.bmp")
    mask = imread_black_and_white(f"{DATA_DIR}/NormalizedMasks/000{i}_00{j}_mano.bmp")
        
    for i2, j2 in ij:
        iris_code2 = imread_black_and_white(f"{DATA_DIR}/IrisCodes/000{i2}_00{j2}_code.bmp")
        mask2 = imread_black_and_white(f"{DATA_DIR}/NormalizedMasks/000{i2}_00{j2}_mano.bmp")
        
        total_mask = mask & mask2 & ap
        total_mask_big = np.array(total_mask.tolist() * 6)
        
        mi, s = match_iris(iris_code, iris_code2, total_mask_big), np.sum(total_mask_big)
        
        ic_ap = iris_code[ap_indices]
        ic2_ap = iris_code2[ap_indices]
        m_ap = total_mask_big[ap_indices]
        
        mi_ap, s_ap = match_iris(ic_ap, ic2_ap, m_ap), np.sum(m_ap)
        
        print(f"000{i}_00{j} - 000{i2}_00{j2}: {mi}, {s}, {mi/s:.2} | {mi_ap}, {s_ap}, {mi_ap/s_ap:.2}")

0000_000 - 0000_000: 0, 978, 0.0 | 0, 423, 0.0
0000_000 - 0000_001: 151, 948, 0.16 | 80, 409, 0.2
0000_000 - 0000_002: 322, 870, 0.37 | 156, 374, 0.42
0000_000 - 0000_003: 287, 882, 0.33 | 151, 378, 0.4
0000_000 - 0001_000: 432, 912, 0.47 | 200, 395, 0.51
0000_000 - 0001_001: 407, 900, 0.45 | 211, 390, 0.54
0000_000 - 0001_002: 390, 876, 0.45 | 195, 379, 0.51
0000_000 - 0001_003: 387, 876, 0.44 | 171, 379, 0.45
0000_001 - 0000_000: 151, 948, 0.16 | 80, 409, 0.2
0000_001 - 0000_001: 0, 1008, 0.0 | 0, 440, 0.0
0000_001 - 0000_002: 319, 858, 0.37 | 171, 370, 0.46
0000_001 - 0000_003: 299, 876, 0.34 | 159, 375, 0.42
0000_001 - 0001_000: 405, 894, 0.45 | 185, 385, 0.48
0000_001 - 0001_001: 394, 894, 0.44 | 181, 386, 0.47
0000_001 - 0001_002: 380, 864, 0.44 | 182, 372, 0.49
0000_001 - 0001_003: 358, 864, 0.41 | 152, 372, 0.41
0000_002 - 0000_000: 322, 870, 0.37 | 156, 374, 0.42
0000_002 - 0000_001: 319, 858, 0.37 | 171, 370, 0.46
0000_002 - 0000_002: 0, 942, 0.0 | 0, 398, 0.0
0000_002 - 0000

In [31]:
DATABASE_ALL = []

for i, j in ij:
    iris_code = imread_black_and_white(f"{DATA_DIR}/IrisCodes/000{i}_00{j}_code.bmp")
    mask = imread_black_and_white(f"{DATA_DIR}/NormalizedMasks/000{i}_00{j}_mano.bmp")
    mask = np.array(total_mask.tolist() * 6)
    
    iris_code = iris_code[ap_indices]
    mask = mask[ap_indices]
    
    DATABASE_ALL.append((iris_code, mask))

In [32]:
import random
random.shuffle(DATABASE_ALL)

In [33]:
len(DATABASE_ALL)

8

In [34]:
DATABASE, DATABASE_TEST = DATABASE_ALL[:4], DATABASE_ALL[4:]

In [35]:
DATABASE_COUNT = len(DATABASE)

In [36]:
import numpy as np
import concrete.numpy as cnp
from functools import reduce

def hamming_distance(x, y) -> int:
    return np.sum(x ^ y)


def min_int(x: int, y: int) -> int:
    """concrete-numpy doesn't yet support min, we have to implement one using
    only supported operations"""
    return (x + y - abs(x - y)) // 2


def min_int_array(a) -> int:
    """Return the minimum value of an array of ints using the `min_int` function"""
    return reduce(min_int, a)

def auth(iris_code, mask):
    """Return the best score from comparing the provided iris and mask to the
    ones in the database, the user fo this function will need to define a threshold
    to decide weather the score means an identified iris or not

    We know this is suboptimal but this is the best we could came up with right now
    as we couldn't figure out a way to return an encrypted ID of the best match or
    a special ID if the score is too low, mainly because we can't compare FHE values
    """
    results = []

    for i in range(DATABASE_COUNT):
        (iris_code2, mask2) = DATABASE[i]
        combined_mask = mask & mask2
        ey1 = iris_code & combined_mask
        ey2 = iris_code2 & combined_mask
        bshd = hamming_distance(ey1, ey2)
        results.append(bshd)

    best_score = min_int_array(results)
    return best_score

In [37]:
class IrisAuthenticator:
    def __init__(self, input_shape, input_max_value) -> None:
        self.inputset = [
            (
                np.random.randint(
                    0, input_max_value + 1, size=input_shape, dtype=np.int64
                ),
                np.random.randint(
                    0, input_max_value + 1, size=input_shape, dtype=np.int64
                ),
            )
            for _ in range(100)
        ]

        self.configuration = cnp.Configuration(
            enable_unsafe_features=True,
            use_insecure_key_cache=True,
            insecure_key_cache_location=".keys",
        )

        self.compiler = cnp.Compiler(
            auth, {"iris_code": "encrypted", "mask": "encrypted"}
        )
        self.circuit = self.compiler.compile(self.inputset, self.configuration)

In [38]:
auth(*DATABASE_TEST[0])

80

In [39]:
input_shape = DATABASE[0][0].shape

In [40]:
authenticator = IrisAuthenticator(input_shape=input_shape, input_max_value=1)

In [41]:
print(authenticator.circuit)

 %0 = iris_code                     # EncryptedTensor<uint1, shape=(1536,)>        ∈ [0, 1]
 %1 = mask                          # EncryptedTensor<uint1, shape=(1536,)>        ∈ [0, 1]
 %2 = [1 1 0 ... 0 1 0]             # ClearTensor<uint1, shape=(1536,)>            ∈ [0, 1]
 %3 = bitwise_and(%1, %2)           # EncryptedTensor<uint1, shape=(1536,)>        ∈ [0, 1]
 %4 = bitwise_and(%0, %3)           # EncryptedTensor<uint1, shape=(1536,)>        ∈ [0, 1]
 %5 = [1 1 0 ... 0 1 1]             # ClearTensor<uint1, shape=(1536,)>            ∈ [0, 1]
 %6 = bitwise_and(%5, %3)           # EncryptedTensor<uint1, shape=(1536,)>        ∈ [0, 1]
 %7 = bitwise_xor(%4, %6)           # EncryptedTensor<uint1, shape=(1536,)>        ∈ [0, 1]
 %8 = sum(%7)                       # EncryptedScalar<uint8>                       ∈ [87, 129]
 %9 = [1 1 0 ... 0 1 0]             # ClearTensor<uint1, shape=(1536,)>            ∈ [0, 1]
%10 = bitwise_and(%1, %9)           # EncryptedTensor<uint1, shape=(1536,)>  

In [ ]:
%%time
authenticator.circuit.encrypt_run_decrypt(*DATABASE_TEST[0])